In [2]:
using LinearAlgebra
using Statistics

In [25]:
# helper methods
function symetric_matrix(dim::Integer)::Matrix
    A = ones(dim,dim);
    for i = 1:dim
        for j = 1:dim
            A[i,j] = rand(1:50);
            A[j,i] = A[i,j];
        end
    end
    return A;
end

function random_matrix(dim::Integer)::Matrix
    A = ones(dim, dim);
    for i = 1:dim
        for j = 1:dim
            A[i,j] = rand(1:50)
        end
    end
    return A
end

function custom_cond(λₘₐₓ, λₘᵢₙ)
    print("TODO")
end

custom_cond (generic function with 1 method)

Poniżej przygotowana jest funkcja do metody iteracji prostej (metoda potęgowa), dzięki której można wyznaczyć największą co do modułu wartość własną, czyli promień spektralny macierzy, który opisany jest wzorem:
## $$\rho(A) = |\lambda_{max}| = \lim_{i \to \infty} \frac{||t_{i+1}||_{\infty}}{||t_{i}||_{\infty}}$$

Przy założeniu, że dowolny wektor początkowy t_{0} != 0, do uzyskania promienia spektralnego prowadzą iteracje postaci:
## $$t_{i+1} = At_{i},\ i=0,1,2...$$



In [8]:
# metoda potegowa
function power_eigen(A::Matrix, iterations::Integer)::Array
    n = size(A, 1);
    X = ones(n,1);
    for i = 1:iterations
        X = A * X;
        X = X / norm(X);
    end

    return X' * A * X / (X' * X);
end

power_eigen (generic function with 1 method)

In [24]:
A = symetric_matrix(5);
@time value = power_eigen(A, 10);
print(value)
e, v = eigen(A)

  0.000055 seconds (27 allocations: 3.219 KiB)
[156.55843075252415]

Eigen{Float64,Float64,Array{Float64,2},Array{Float64,1}}
values:
5-element Array{Float64,1}:
 -26.96040326411249
 -11.25335022934285
   5.8855815647038305
  31.76974117622714
 156.55843075252423
vectors:
5×5 Array{Float64,2}:
 -0.797682    0.274998   0.0589095   0.246338  -0.473208
  0.425915    0.225063  -0.636115    0.377665  -0.469756
 -0.122094    0.182555  -0.393763   -0.871913  -0.191009
 -0.0544509  -0.884551  -0.0453287  -0.056879  -0.45751
  0.405497    0.240762   0.659381   -0.18226   -0.55642

Jak widać z wyniku wykonania powyższego kodu, uzyskaliśmy największą co do modułu wartość własną macierzy. Ta wartość zgadza sięz wartością otrzymaną za pomocą metody eigen(A). Za pomocą metody potęgowej można również wyznaczyć najmniejszą co do modułu wartość własną macierzy. Korzysta się z twierdzenia dotyczącego przesunięcia spektrum macierzy:<br />
### "Jeżeli $\lambda$ jest wartością własną macierzy A, to $\lambda + r$ jest wartością własną macierzy $A + \tau I$"
Po zastosowaniu przesunięcia $B = A - \lambda_{max}I$ można ponownie zastosować metodę iteracji prostej dla macierzy B zbieżną do największej co do modułu wartości własnej $\lambda = \ambda_{min}-\lambda_{max}$. Stąd można wyznaczyć $\lambda_{min}$.

In [ ]:
# TODO wyznaczenie lambda min

Mając największą i najmniejszą wartość własną macierzy, można wyznaczyć współczynnik uwarunkowania macierzy:
$$cond(A)=\sqrt{\frac{\lambda_{max}(A^TA)}{\lambda_{min}(A^TA)}}$$
We wzorze przyjęto, że macierz A jest symetryczna, co oznacza, że $A=A^T$, ewentualnie można macierz wejściową doprowadzić do postaci symetrycznej stosując wzór $B = A^TA$

In [ ]:
# TODO cond(A) #helper method